In [ ]:
from src.utils import data_load
import pandas as pd
from src.s3_utils import pandas_from_csv_s3
import re

# Data processing: Join PHQ9, GAD7 and ACE datasets together by record_id and redcap_event_name

In [ ]:
data = data_load(data_keys={'phq9', 'generalized_anxiety_disorder_scale_gad7', 'ace', 'surveys', 'study_ids', 'check_in_adherence_log'})

In [ ]:
outcomes = pd.merge(data['phq9'], data['generalized_anxiety_disorder_scale_gad7'],  how='outer', left_on=['record_id','redcap_event_name'], right_on = ['record_id','redcap_event_name'])

In [ ]:
overall_df = pd.merge(data['ace'].drop(columns=['redcap_event_name']).dropna(), outcomes, how='left', on='record_id')

# Convert redcap_event_name to date for PHQ9, GAD7 and ACE datasets

In [ ]:
# read study ids
id_df = data['study_ids'][['record_id', 'evidation_id']]
id_df.rename(columns={'evidation_id': 'user_id'}, inplace=True)

# add ids to survey
overall_df = overall_df.merge(id_df, on=['record_id'])
overall_df.user_id = overall_df.user_id.fillna(-1).astype(int)

# standarize naming convention for easier processing later on
overall_df.redcap_event_name = overall_df.redcap_event_name.replace('postnatal_checkin_arm_1','postnatal_ci_1_arm_1')

In [ ]:
# read check-in dates
ci_df = data['check_in_adherence_log']
cols = ['record_id'] + [col for col in ci_df.columns if '_date' in col]
ci_df = ci_df[cols]

# standarize naming convention for easier processing later on
ci_df = ci_df.rename(columns={'checkin_postnatal_date': 'checkin_postnatal_date_1'})

# add dates to survey, need to map it using the check_in_adherence_log
def conver_checkin_string(x):
    x = x.split('_arm')[0] #delete all characters after the word 'arm'
    num = int(re.search(r'\d+', x).group())
    if 'postnatal' in x:
        return f'checkin_postnatal_date_{num}'
    else:
        return f'checkin_{num}_date'

# map checkin_postnature_date_{num} OR checkin_{num}_date to the actual date
def map_date(x):
    checkin_string_col = x['checkin_string']
    return x[checkin_string_col]

overall_df = overall_df.merge(ci_df, on=['record_id'])
overall_df['checkin_string'] = overall_df.redcap_event_name.apply(conver_checkin_string)
overall_df['date'] = overall_df.apply(map_date, axis=1)
overall_df = overall_df[overall_df.columns.drop(list(overall_df.filter(regex='checkin_')))]
overall_df['date'] = pd.to_datetime(overall_df['date'])

In [ ]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', None)
overall_df.loc[overall_df['record_id'] == 28][['date', 'redcap_event_name']] #check if dates are correctly processed

# Add Global survey data - PROMIS quality of life

In [ ]:
promis_survey = data['surveys']
promis_survey = promis_survey.loc[promis_survey['question_id'] == 121]
promis_survey['date'] = pd.to_datetime(promis_survey['date'])

In [ ]:
promis_survey['question_text'].unique()

In [ ]:
promis_survey['answer_text'].unique()

In [ ]:
promis_survey.loc[promis_survey['user_id'] == 28]

# Process PHQ9, GAD and PROMIS data by taking the average over time for each individual

In [ ]:
# # more balanced set up
# ace_levels = {
#     (0, 1) : 0,
#     (2, 10): 1
# }
# phq9_levels = {
#     (0, 4) : 0,
#     (5, 9): 1,
#     (10, 14): 1,
#     (15, 19): 1,
#     (20, 27): 1
# }
# gad_levels = {
#     (0, 4) : 0,
#     (5, 9): 1,
#     (10, 14): 1,
#     (15, 21): 1
# }
# promis_levels = {
#     "Always" : 1,
#     "Often": 1,
#     "Sometimes": 1,
#     "Rarely": 0,
#     "Never": 0
# }

# from literature
ace_levels = {
    (0, 4) : 0,
    (5, 10): 1
}
phq9_levels = {
    (0, 4) : 0,
    (5, 9): 1,
    (10, 14): 2,
    (15, 19): 3,
    (20, 27): 4
}
gad_levels = {
    (0, 4) : 0,
    (5, 9): 1,
    (10, 14): 2,
    (15, 21): 3
}
promis_levels = {
    "Always" : 4,
    "Often": 3,
    "Sometimes": 2,
    "Rarely": 1,
    "Never": 0
}
def map_levels(x, map_dict):
    for key in map_dict:
        if isinstance(x, str):
            if x == key:
                return map_dict[key]
        else:
            if x >= key[0] and x <= key[1]:
                return map_dict[key]

In [ ]:
processed_overall_df = pd.DataFrame(columns=['user_id', 'ace_sum', 'phq9_sum', 'gad_sum', 'promis_mean'])
for uid in overall_df['user_id'].unique():
    each_df = overall_df.loc[overall_df['user_id']==uid]
    ace_lst = [f'ace_{x}' for x in range(1,11)]
    phq9_lst = [f'phq9_{x}' for x in range(1,11)]
    gad_lst = [f'gad_{x}' for x in range(1,9)]
    ace_sum = each_df[ace_lst].sum(axis=1)
    ace_sum_mean = ace_sum.apply(map_levels, map_dict=ace_levels).mean()
    phq9_sum = each_df[phq9_lst].sum(axis=1)
    phq9_sum_mean = phq9_sum.apply(map_levels, map_dict=phq9_levels).mean()
    gad_sum = each_df[gad_lst].sum(axis=1)
    gad_sum_mean = gad_sum.apply(map_levels, map_dict=gad_levels).mean()
    each_promis_df = promis_survey.loc[promis_survey['user_id']==uid]['answer_text'].apply(map_levels, map_dict=promis_levels)
    promis_mean = each_promis_df.mean()
    processed_overall_df = processed_overall_df.append({'user_id': uid, 'ace_sum': ace_sum_mean, 'promis_mean': promis_mean, 'phq9_sum': phq9_sum_mean, 'gad_sum': gad_sum_mean}, ignore_index=True)

In [ ]:
processed_overall_df = processed_overall_df.dropna()

In [ ]:
# processed_overall_df.to_csv('processed_causal_ace_4nodes_ref_levels_reverse.csv', index=False)

In [ ]:
# processed_overall_df[['phq9_sum', 'gad_sum', 'promis_mean']] = processed_overall_df[['phq9_sum', 'gad_sum', 'promis_mean']] * 5

In [ ]:
processed_overall_df['ace_sum'].hist()

In [ ]:
processed_overall_df['phq9_sum'].round().hist()

In [ ]:
processed_overall_df['gad_sum'].round().hist()

In [ ]:
processed_overall_df['promis_mean'].round().hist()

In [ ]:
processed_overall_df.head()

In [ ]:
import networkx
import notears.notears as notears
import matplotlib.pyplot as plt

data = processed_overall_df[['ace_sum', 'phq9_sum', 'gad_sum', 'promis_mean']].to_numpy().tolist()
output_dict = notears.run(notears.notears_standard, data, notears.loss.least_squares_loss, notears.loss.least_squares_loss_grad, e=1e-8, verbose=False)

In [ ]:
print('Acyclicity loss: {}'.format(output_dict['h']))
print('Least squares loss: {}'.format(output_dict['loss']))

In [ ]:
plt.matshow(output_dict['W'])
plt.title("Learned adjacency matrix")
plt.colorbar()

In [ ]:
acyclic_W = notears.utils.threshold_output(output_dict['W'])

In [ ]:
plt.matshow(acyclic_W)
plt.title("Learned adjacency matrix (thresholded)")
plt.colorbar()

In [ ]:
G = networkx.DiGraph(acyclic_W)
networkx.draw(G, with_labels=True)


In [ ]:
weighted_G = networkx.DiGraph((output_dict['W'] * acyclic_W).round(1))
layout = networkx.spring_layout(weighted_G)
networkx.draw(weighted_G, layout, node_size=1000, with_labels=True, font_weight='bold',    font_size=15)
labels = networkx.get_edge_attributes(weighted_G,'weight')
networkx.draw_networkx_edge_labels(weighted_G,pos=layout,edge_labels=labels)
plt.show()